---
layout: default
title: "Deep Learning - Speed up learning - Gradient Checking"
categories: deeplearning
permalink: /ML29/
order: 29
comments: true
---

In [2]:
%pylab --no-import-all inline
from plot_ann import ann
from scipy.stats import multivariate_normal

Populating the interactive namespace from numpy and matplotlib


# Gradient Checking
Gradient checking is a very powerful technique that allow to diagnose problems in the calculation of gradients

## Numerical approximation of gradients
